In [ ]:
# Authors: Vasisht Duddu, Anudeep Das, Nora Khayata, Hossein Yalame, Thomas Schneider, N Asokan
# Copyright 2024 Secure Systems Group, University of Waterloo & Aalto University, https://crysp.uwaterloo.ca/research/SSG/
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import argparse
import logging
from pathlib import Path
from typing import Optional
from tqdm import tqdm
import os
import torch
from torch.utils.data import DataLoader
from ogb.nodeproppred import Evaluator
from timeit import Timer

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import jupyter_utils

from ogb.nodeproppred import Evaluator


/home/a38das/miniconda3/envs/property-attestation/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch


In [ ]:
def handle_args() -> argparse.Namespace:
    parser: argparse.ArgumentParser = argparse.ArgumentParser()

    parser.add_argument('--raw_path', type=str, default="../dataset/", help='Root directory of the dataset')
    parser.add_argument('--result_path', type=str, default="results_overlap_25/", help='Results directory for the models')
    parser.add_argument('--dataset', type=str, default="CENSUS", help='Options: CENSUS, BONEAGE, ARXIV')
    parser.add_argument('--sampled_dataset_save_path', type=str, default="sampled_datasets/", help='Directory location of where to save the training and testing data')
    parser.add_argument('--filter', type=str,required=False,help='while filter to use')
    parser.add_argument('--ratio', type=float, default=0.5,help='what ratio of the new sampled dataset')
    parser.add_argument('--num', type=int, default=1,help='how many classifiers to train?')
    parser.add_argument('--split', choices=["verifier", "prover"],default="prover",help='which split of data to use')
    parser.add_argument('--test', action='store_true')
    parser.add_argument('--overlap', type=float, default=0., help='How much overlap to use between the prover and verifier dataset')

    parser.add_argument('--num_layers', type=int, default=3)
    parser.add_argument('--hidden_channels', type=int, default=256)
    parser.add_argument('--dropout', type=float, default=0.5)
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--decay', type = float, default = 0, help = "Weight decay/L2 Regularization")
    parser.add_argument('--epochs', type=int, default=100)
    parser.add_argument('--degree', type=float, default=13)
    parser.add_argument('--prune', type=float, default=0)

    # Notebook specific args
    parser.add_argument('--ip', default='')
    parser.add_argument('--stdin', default='9035')
    parser.add_argument('--control', default='9033')
    parser.add_argument('--hb', default='9032')
    parser.add_argument('--Session.signature_scheme', default="hmac-sha256")
    parser.add_argument('--Session.key', default='')
    parser.add_argument('--shell', default=9034)
    parser.add_argument('--transport', default='tcp')
    parser.add_argument('--iopub', default=9036)

    args: argparse.Namespace = parser.parse_args()
    return args

In [3]:
logging.basicConfig(level=logging.INFO, filename="generate_models.log", filemode="w")
log: logging.Logger = logging.getLogger("GenerateModels")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
def initialize_training(args: argparse.Namespace, log: logging.Logger) -> None:

    raw_path: Path = Path(args.raw_path)
    raw_path_status: Optional[Path] = jupyter_utils.create_dir_if_doesnt_exist(raw_path, log)
    result_path: Path = Path(args.result_path)
    result_path_status: Optional[Path] = jupyter_utils.create_dir_if_doesnt_exist(result_path, log)
    suff = str(args.ratio)
    if args.dataset == 'CENSUS':
        suff = args.filter +"/"+ suff
    elif args.dataset == 'ARXIV':
        suff = str(args.degree)
    if args.test:
        suff = "test" + "/"+suff
    sampled_save_path: Path = Path(args.sampled_dataset_save_path) / args.dataset / args.split / suff
    sampled_save_path_status, sampled_save_path_already_exists = jupyter_utils.create_dir_if_doesnt_exist(sampled_save_path, log)
    if (raw_path_status or result_path_status or sampled_save_path_status) is None:
        msg: str = f"Something went wrong when creating {raw_path} or {result_path} or {sampled_save_path}. Aborting..."
        log.error(msg)
        raise EnvironmentError(msg)
    log.info("Dataset {}".format(args.dataset))


    if args.dataset == "CENSUS":
        raw_path = raw_path / "census_splits"
        result_path = result_path / "CENSUS"
        dataset = jupyter_utils.CensusWrapper(path=raw_path,filter_prop=args.filter, ratio=float(args.ratio), split=args.split)
        dataset_prover = None
        if args.overlap > float(0):
            dataset_prover = jupyter_utils.CensusWrapper(path=raw_path,filter_prop=args.filter, ratio=float(args.ratio), split="prover")
        

        for i in tqdm(range(1, args.num + 1)):
            print("Training classifier %d" % i)

            # if sampled_save_path_already_exists:
            #     (x_tr, y_tr), (x_te, y_te), _, _ = data.load_saved_traintest_data(args, sampled_save_path)
            # else:
            # Putting it back inside
            if dataset_prover is not None:
                _, _, _, train_df_prover, test_df_prover = dataset_prover.load_data(send_original_dfs=True)
                (x_tr, y_tr), (x_te, y_te), cols, train_df, test_df = jupyter_utils.generate_overlap_datasets(args, dataset, dataset_prover)
            else:
                (x_tr, y_tr), (x_te, y_te), cols, train_df, test_df = dataset.load_data(send_original_dfs=True)

            jupyter_utils.save_sampled_dataset(args, sampled_save_path, (x_tr, y_tr), (x_te, y_te), training_df_census = train_df, testing_df_census = test_df, suffix = i)
            print(f'x_tr shape: {x_tr.shape}')
            print(f'x_te shape: {x_te.shape}')

            clf = jupyter_utils.get_model(x_tr.shape[1]).to(device)
            train_data, val_data = (x_tr, y_tr), (x_te, y_te)
            ds_train = jupyter_utils.CensusSet(*train_data, squeeze=False)
            ds_val = jupyter_utils.CensusSet(*val_data, squeeze=False)
            train_loader = DataLoader(
                ds_train,
                batch_size=args.batch_size,
                shuffle=False,
                # num_workers=num_workers,
                # worker_init_fn=utils.worker_init_fn,
                # #pin_memory=True,
                # prefetch_factor=prefetch_factor
            )
            val_factor = 1
            test_loader = DataLoader(
                ds_val,
                batch_size=args.batch_size * val_factor,
                shuffle=False,
                # num_workers=num_workers,
                # worker_init_fn=utils.worker_init_fn,
                # #pin_memory=True,
                # prefetch_factor=prefetch_factor
            )
            
            # clf.fit(x_tr, y_tr.ravel())
            # train_acc = 100 * clf.score(x_tr, y_tr.ravel())
            # test_acc = 100 * clf.score(x_te, y_te.ravel())
            # print("Classifier %d : Train acc %.2f , Test acc %.2f\n" % (i, train_acc, test_acc))
            _, vacc = jupyter_utils.boneage_train(clf, (train_loader, test_loader),lr=args.lr,epoch_num=args.epochs,weight_decay=args.decay)

            if args.test:
                save_path = jupyter_utils.get_models_path(result_path, os.path.join(args.split,"test"), args.filter, str(args.ratio))
            else:
                save_path = jupyter_utils.get_models_path(result_path, args.split, args.filter, str(args.ratio))
            if not os.path.isdir(save_path):
                os.makedirs(save_path)
            print("Saving model to %s" % os.path.join(save_path, str(i) + "_%.2f" % vacc))
            jupyter_utils.save_model_census(clf, os.path.join(save_path,str(i) + "_%.2f" % vacc))



    elif args.dataset == "BONEAGE":
        result_path = result_path / "BONEAGE"
        raw_path = raw_path / "boneage_splits"

        def filter1(x): return x["gender"] == 1

        df_train, df_val = jupyter_utils.get_df(raw_path, args.split)
        df_train_prover, df_val_prover = None, None
        features = jupyter_utils.get_features(raw_path, args.split)
        features_prover = None
        if args.overlap > float(0):
            df_train_prover, df_val_prover = jupyter_utils.get_df(raw_path, "prover")
            features_prover = jupyter_utils.get_features(raw_path, "prover")
            features['train'] = {**features['train'], **features_prover['train']}
            features['val'] = {**features['val'], **features_prover['val']}

        # print(f'features are: {features}')

        n_train, n_test = 700, 200
        if args.split == "prover":
            n_train, n_test = 1400, 400

        
        for i in range(args.num):
            if jupyter_utils.check_if_exists(result_path, i+1, args.split, args.ratio):
                continue
            # if sampled_save_path_already_exists:
            #     df_train_processed, df_val_processed = data.load_saved_traintest_data(args, sampled_save_path)
            # else:
            if df_train_prover is not None:
                df_train_processed, df_val_processed = jupyter_utils.generate_overlap_datasets(args, (df_train, df_val), (df_train_prover, df_val_prover))
            else:
                df_train_processed = jupyter_utils.heuristic(df_train, filter1, args.ratio,n_train, class_imbalance=1.0, n_tries=300)
                df_val_processed = jupyter_utils.heuristic(df_val, filter1, args.ratio,n_test, class_imbalance=1.0, n_tries=300)

            jupyter_utils.save_sampled_dataset(args, sampled_save_path, df_train_processed, df_val_processed, suffix=i)

            model = jupyter_utils.BoneModel(1024).to(device)
            ds = jupyter_utils.BoneWrapper(df_train_processed,df_val_processed,features=features,augment=False)
            train_loader, val_loader = ds.get_loaders(256*32, shuffle=True)
            _, vacc = jupyter_utils.boneage_train(model, (train_loader, val_loader),lr=1e-3,epoch_num=100,weight_decay=1e-4)

            if args.test:
                model_dir_path = os.path.join(result_path, args.split, "test", str(args.ratio))
            else:
                model_dir_path = os.path.join(result_path, args.split, str(args.ratio))

            if not os.path.exists(model_dir_path):
                os.makedirs(model_dir_path)

            if args.test:
                jupyter_utils.save_model_boneage(result_path, model, os.path.join(args.split,"test"), args.ratio,"%d_%.2f.pth" %(i+1, vacc*100))
            else:
                jupyter_utils.save_model_boneage(result_path, model, args.split, args.ratio,"%d_%.2f.pth" %(i+1, vacc*100))
            print("[Status] %d/%d" % (i+1, args.num))


    elif args.dataset == "ARXIV":
        result_path = result_path / "ARXIV"
       
        assert args.prune >= 0 and args.prune <= 0.1 # Prune ratio should be valid and not too large
        ds = jupyter_utils.ArxivNodeDataset(args.split)
        # Need to save this one

        n_edges = ds.g.number_of_edges()
        n_nodes = ds.g.number_of_nodes()
        print("""----Data statistics------'\n #Edges %d; #Nodes %d; #Classes %d""" %(n_edges, n_nodes, ds.num_classes,))

        print("-> Before modification")
        print("Nodes: %d, Average degree: %.2f" %(ds.num_nodes, ds.g.number_of_edges() / ds.num_nodes))
        print("Train: %d, Test: %d" % (len(ds.train_idx), len(ds.test_idx)))

        ds.change_mean_degree(args.degree, args.prune)

        pick_tr, pick_te = 30000, 10000
        if args.split == "prover":
            pick_tr, pick_te = 62000, 35000

        ds.label_ratio_preserving_pick(pick_tr, pick_te)

        print("-> After modification")
        print("Nodes: %d, Average degree: %.2f" %(ds.num_nodes, ds.g.number_of_edges() / ds.num_nodes))
        print("Train: %d, Test: %d" % (len(ds.train_idx), len(ds.test_idx)))

        for index in range(args.num):
            model = jupyter_utils.get_model_arxiv(ds, args)
            evaluator = Evaluator(name='ogbn-arxiv')

            run_accs = jupyter_utils.train_model_arxiv(ds, model, evaluator, args)
            acc_tr, acc_te = run_accs["train"][-1], run_accs["test"][-1]
            print("Train accuracy: {:.2f}; Test accuracy: {:.2f}".format(acc_tr*100,acc_te*100))

            if args.test:
                savepath = result_path / args.split / "test" / str(int(args.degree))
            else:
                savepath = result_path / args.split / str(int(args.degree)) 
            savepath_status: Optional[Path] = jupyter_utils.create_dir_if_doesnt_exist(savepath, log)
            jupyter_utils.save_model_arxiv(model, savepath / "{}_{:.2f}.pth".format(index,acc_tr*100))

    else:
        msg_dataset_error: str = "No such dataset"
        log.error(msg_dataset_error)
        raise EnvironmentError(msg_dataset_error)

In [6]:
 # You can see the required arguments in generate_models.sh
args = handle_args()

In [7]:
def train_all_ratios(args, ratios):
    for ratio in ratios:
        args.ratio = ratio
        initialize_training(args, log)

In [8]:
census_ratios = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [9]:
args.dataset = 'CENSUS'
args.filter = 'sex'
args.split = 'verifier'  
args.num = 1000

train_all_ratios(args, census_ratios)

args.split = 'prover'
train_all_ratios(args, census_ratios)

  0%|          | 0/1000 [00:00<?, ?it/s]

Training classifier 1
Datasets saved
x_tr shape: (4400, 42)
x_te shape: (2000, 42)










































































[Train] Epoch 1, Loss: 0.72658, Acc: 0.7093: 100%|██████████| 69/69 [00:00<00:00, 220.06it/s]


[Validation], Loss: 0.54825, Accuracy: 0.7515










































































[Train] Epoch 2, Loss: 0.52759, Acc: 0.7578: 100%|██████████| 69/69 [00:00<00:00, 330.15it/s]


...

[Validation], Loss: 0.39980, Accuracy: 0.8203









































































[Train] Epoch 77, Loss: 0.37906, Acc: 0.8270: 100%|██████████| 69/69 [00:00<00:00, 353.74it/s]


[Validation], Loss: 0.40101, Accuracy: 0.8213






















































[Train] Epoch 78, Loss: 0.37287, Acc: 0.8266:  72%|███████▏  | 50/69 [00:00<00:00, 352.45it/s]
train_acc: 0.83 | val_acc: 0.82 |:  77%|███████▋  | 77/100 [00:17<00:05,  4.48it/s]
  0%|          | 2/1000 [01:03<8:48:10, 31.75s/it]


KeyboardInterrupt: 

In [10]:
args.filter = 'race'
args.split = 'verifier'
train_all_ratios(args, census_ratios)

args.split = 'prover'
train_all_ratios(args, census_ratios)

  0%|          | 0/1000 [00:00<?, ?it/s]

Training classifier 1
Datasets saved
x_tr shape: (2372, 42)
x_te shape: (1167, 42)










































[Train] Epoch 1, Loss: 0.62006, Acc: 0.7500: 100%|██████████| 38/38 [00:00<00:00, 336.63it/s]


[Validation], Loss: 0.33297, Accuracy: 0.8573










































[Train] Epoch 2, Loss: 0.27863, Acc: 0.8808: 100%|██████████| 38/38 [00:00<00:00, 355.93it/s]


...

[Validation], Loss: 0.30023, Accuracy: 0.8696










































[Train] Epoch 43, Loss: 0.26649, Acc: 0.8865: 100%|██████████| 38/38 [00:00<00:00, 317.56it/s]


[Validation], Loss: 0.34400, Accuracy: 0.8521

































[Train] Epoch 44, Loss: 0.28886, Acc: 0.8766:  79%|███████▉  | 30/38 [00:00<00:00, 348.30it/s]
train_acc: 0.89 | val_acc: 0.85 |:  43%|████▎     | 43/100 [00:05<00:07,  7.55it/s]
  0%|          | 4/1000 [00:56<3:54:48, 14.14s/it]


KeyboardInterrupt: 

In [14]:
args.dataset = 'BONEAGE'
args.split = 'verifier'  
args.num = 600

boneage_ratios = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
train_all_ratios(args, boneage_ratios)

args.split = 'prover'
train_all_ratios(args, boneage_ratios)

0.2000: 100%|██████████| 300/300 [00:00<00:00, 673.80it/s]


Datasets saved


[Train] Epoch 1, Loss: 0.69656, Acc: 0.4800: 100%|██████████| 1/1 [00:00<00:00,  7.70it/s]
train_acc: 0.48 | val_acc: 0.50 |:   1%|          | 1/100 [00:00<00:23,  4.22it/s]

[Validation], Loss: 0.68269, Accuracy: 0.5025


[Train] Epoch 2, Loss: 0.68741, Acc: 0.5036: 100%|██████████| 1/1 [00:00<00:00,  7.92it/s]
train_acc: 0.50 | val_acc: 0.51 |:   2%|▏         | 2/100 [00:00<00:22,  4.32it/s]

...

[Train] Epoch 92, Loss: 0.26391, Acc: 0.8907: 100%|██████████| 1/1 [00:00<00:00,  7.71it/s]
train_acc: 0.89 | val_acc: 0.82 |:  92%|█████████▏| 92/100 [00:21<00:01,  4.31it/s]

[Validation], Loss: 0.39395, Accuracy: 0.8175


[Train] Epoch 93, Loss: 0.27817, Acc: 0.8757: 100%|██████████| 1/1 [00:00<00:00,  7.63it/s]
train_acc: 0.88 | val_acc: 0.81 |:  93%|█████████▎| 93/100 [00:21<00:01,  4.30it/s]

[Validation], Loss: 0.40250, Accuracy: 0.8100


[Train] Epoch 94, Loss: 0.27258, Acc: 0.8743: 100%|██████████| 1/1 [00:00<00:00,  7.69it/s]


In [13]:
args.dataset = 'ARXIV'
args.split = 'verifier'  
args.num = 800

arxiv_ratios = [9, 10, 11, 12, 13, 14, 15, 16, 17]
train_all_ratios(args, arxiv_ratios)

args.split = 'prover'
train_all_ratios(args, arxiv_ratios)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:06<00:00, 13.25it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 16980.99it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00, 241.00it/s]

...

Train accuracy: 74.42; Test accuracy: 73.98


Epoch: 100, Loss: 0.9108, Train: 74.36%, Test: 74.06%: 100%|██████████| 100/100 [00:09<00:00, 10.66it/s]


Train accuracy: 74.36; Test accuracy: 74.06


Epoch: 37, Loss: 1.1114, Train: 69.95%, Test: 70.09%:  37%|███▋      | 37/100 [00:03<00:06, 10.45it/s]


KeyboardInterrupt: 